'''<br>
    @Author: v sanjay kumar<br>
    @Date: 2024-09-06 04:00:30<br>
    @Last Modified by: v sanjay kumar<br>
    @Last Modified time: 2024-09-07 04:00:30.<br>
    @Title :All covid problems in pyspark <br>

'''


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, round
from pyspark.sql.functions import coalesce, col


1.find death percentage  globaly  and localy

In [4]:
try:
    spark = SparkSession.builder \
        .appName("Find the global") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\covid_19_clean_complete.csv"
    df_global = spark.read.csv(file_path, header=True, inferSchema=True)
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\Covid Problem\covid_19_india.csv"
    df_local = spark.read.csv(file_path, header=True, inferSchema=True)
    df_local.createOrReplaceTempView("covid_19_india")
    df_global.createOrReplaceTempView("country")
    global_result = spark.sql('''
    SELECT `Country/Region`, 
           ROUND((SUM(Deaths) * 100.0 / SUM(Confirmed)), 4) AS Death_percentage
    FROM country
    GROUP BY `Country/Region`
''')
   
    local_result=spark.sql('''
    SELECT `State/UnionTerritory`, 
           ROUND((SUM(CAST(Deaths AS INT)) * 100.0) / SUM(CAST(Confirmed AS INT)), 2) AS Death_percentage
    FROM covid_19_india
    GROUP BY `State/UnionTerritory`
    ORDER BY `State/UnionTerritory`
''')
    print("---------global percentage ---------")
    global_result.show()
    print("----------local_percentage-----------")
    local_result.show()
    spark.stop()

except Exception as e:
    print(e)

---------global percentage ---------
+--------------+----------------+
|Country/Region|Death_percentage|
+--------------+----------------+
|          Chad|          8.5993|
|      Paraguay|          1.0635|
|        Russia|          1.3640|
|         Yemen|         26.3575|
|       Senegal|          1.5353|
|    Cabo Verde|          1.0322|
|        Sweden|          9.0267|
|        Guyana|          7.0512|
|       Eritrea|          0.0000|
|   Philippines|          3.7305|
|         Burma|          2.5369|
|      Djibouti|          0.8956|
|      Malaysia|          1.4792|
|     Singapore|          0.0697|
|          Fiji|          0.0000|
|        Turkey|          2.6032|
|        Malawi|          1.8290|
|Western Sahara|          6.9922|
|          Iraq|          3.9239|
|       Germany|          4.1375|
+--------------+----------------+
only showing top 20 rows

----------local_percentage-----------
+--------------------+----------------+
|State/UnionTerritory|Death_percentage|
+--

2. To find out the infected population percentage locally and globally


In [10]:
try:
    # Start Spark session
    spark = SparkSession.builder \
        .appName("Find the infected population percentage") \
        .getOrCreate()

    # Load local dataset (India specific)
    file_path_local = r"file:///C:/Users/sanju/OneDrive/Documents/dataset/Covid Problem/covid_19_india.csv"
    df = spark.read.csv(file_path_local, header=True, inferSchema=True)
    df.createOrReplaceTempView("country")
    global_infected=spark.sql('''
    SELECT 
    `State/UnionTerritory`, 
        SUM(`Confirmed`) AS Total_Cases, 
        SUM(`ConfirmedIndianNational`) AS Total_Active,
        (SUM(`ConfirmedIndianNational`) * 100.0 / SUM(`Confirmed`)) AS Globally_Infected_percentage
    FROM 
        country
    GROUP BY 
        `State/UnionTerritory` ''')
    localy_infected=spark.sql('''
    SELECT 
    `State/UnionTerritory`, 
    SUM(`Confirmed`) AS Total_Confirmed, 
    SUM(`Cured`) AS Total_Cured,
    CASE 
        WHEN SUM(`Confirmed`) > 0 THEN 
            100 - (SUM(`Cured`) * 100.0 / SUM(`Confirmed`))
        ELSE 
            NULL 
    END AS locally_Infected
    FROM 
        country
    GROUP BY 
        `State/UnionTerritory`
''')
    print("-----------globla infected percentage----------")
    global_infected.show()
    print("-----------local infected percentage-----------")
    localy_infected.show()
    spark.stop()


except Exception as e:
    print(e)   


-----------globla infected percentage----------
+--------------------+-----------+------------+----------------------------+
|State/UnionTerritory|Total_Cases|Total_Active|Globally_Infected_percentage|
+--------------------+-----------+------------+----------------------------+
|            Nagaland|    5041742|        NULL|                        NULL|
|           Karnataka|  485970693|       405.0|        8.333835884214524E-5|
|              Odisha|  160130533|        23.0|        1.436328198570350...|
|              Kerala|  458906023|      1091.0|        2.377393072480986E-4|
|              Ladakh|    4054293|       162.0|         0.00399576448964098|
|Dadra and Nagar H...|    1938632|        NULL|                        NULL|
|          Tamil Nadu|  431928644|       138.0|        3.194972176932076...|
|           Telengana|   69990668|       246.0|        3.514754281242179...|
|        Chhattisgarh|  163776262|        25.0|        1.526472743650725...|
|      Maharashtra***|    62

3.The countries with the highest infection rates


In [28]:
try:
    spark = SparkSession.builder \
        .appName("Find the global and local death percentage") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\country_wise_latest.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    # df.show()
    df.createOrReplaceTempView("country")
    result = spark.sql('''
    SELECT `Country/Region`, 
           ROUND((CAST(`Confirmed` AS FLOAT) / (SELECT SUM(CAST(`Confirmed` AS FLOAT)) FROM country)) * 100, 2) AS Infection_Rate_Percentage
    FROM country
    GROUP BY `Country/Region`, `Confirmed`
    ORDER BY Infection_Rate_Percentage DESC
''')

    result.show(10)
    
    spark.stop()

except Exception as e:
    print(e)


root
 |-- Country/Region: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- Deaths / 100 Cases: double (nullable = true)
 |-- Recovered / 100 Cases: double (nullable = true)
 |-- Deaths / 100 Recovered: string (nullable = true)
 |-- Confirmed last week: integer (nullable = true)
 |-- 1 week change: integer (nullable = true)
 |-- 1 week % increase: double (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------------------+
|Country/Region|Infection_Rate_Percentage|
+--------------+-------------------------+
|            US|                    26.03|
|        Brazil|                    14.82|
|         India|                     8.98|
|        Russia|                     4.96|
|  South

4.The countries and continents with the highest death counts


In [30]:
try:
    spark = SparkSession.builder \
        .appName("Find hight death counts") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\country_wise_latest.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    # df.show()
    df.createOrReplaceTempView("country")
    result = spark.sql('''
        
    SELECT `Country/Region`,`WHO Region`, Deaths
    FROM country
    ORDER BY Deaths DESC;
    
''')

    result.show(10)
    
    spark.stop()

except Exception as e:
    print(e)


root
 |-- Country/Region: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- Deaths / 100 Cases: double (nullable = true)
 |-- Recovered / 100 Cases: double (nullable = true)
 |-- Deaths / 100 Recovered: string (nullable = true)
 |-- Confirmed last week: integer (nullable = true)
 |-- 1 week change: integer (nullable = true)
 |-- 1 week % increase: double (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+--------------------+------+
|Country/Region|          WHO Region|Deaths|
+--------------+--------------------+------+
|            US|            Americas|148011|
|        Brazil|            Americas| 87618|
|United Kingdom|              Europe| 45844|
|        Mexico|            Americas| 4

5.Average number of deaths by day (Continents and Countries) 

In [6]:
try:
    spark = SparkSession.builder \
        .appName("Avarage number of deaths") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\worldometer_data.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    df.createOrReplaceTempView("worldometer_data")
    result = spark.sql('''
    SELECT
        `Country/Region`,
        Population,
        TotalCases,
        CASE 
            WHEN Population = 0 THEN NULL
            ELSE CAST(TotalCases AS FLOAT) / CAST(Population AS FLOAT)
        END AS CasesPerPopulation
    FROM
        worldometer_data
    WHERE
        Population RLIKE '^[0-9]+$'
        AND TotalCases RLIKE '^[0-9]+$'
        AND CAST(Population AS FLOAT) <> 0
    ORDER BY
        CasesPerPopulation DESC ''')
    result.show(10)

    spark.stop()


except Exception as e:
    print(e)

root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDe

 6.Average of cases divided by the number of population of each country (TOP 10)

In [12]:
try:
    spark = SparkSession.builder \
        .appName("Avarage of cases divided by the population") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\worldometer_data.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    df.createOrReplaceTempView("worldometer_data")
    result=spark.sql('''
    SELECT `Country/Region`,
           CAST(Population AS FLOAT) AS Population,
           CAST(TotalCases AS FLOAT) AS TotalCases,
           CASE 
               WHEN CAST(Population AS FLOAT) = 0 THEN NULL
               ELSE CAST(TotalCases AS FLOAT) / CAST(Population AS FLOAT)
           END AS CasesPerPopulation
    FROM worldometer_data
    WHERE Population RLIKE '^[0-9]+$'
      AND TotalCases RLIKE '^[0-9]+$'
      AND CAST(Population AS FLOAT) <> 0
    ORDER BY CasesPerPopulation DESC
''')
    result.show(10)

    spark.stop()


except Exception as e:
    print(e)


root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+------------+----------+--------------------+
|Country/Region|  Population|TotalCases|  CasesPerPopulation|
+--------------+------------+----------+--------------------+
|         Qatar|   2807805.0|  112092.0|0.039921575750452756|
| French Guiana|    299385.0|    8127